#### LeetCode - SQL - #178

Refer [178. Rank Scores](https://leetcode.com/problems/rank-scores/description/)

Write a solution to find the rank of the scores. The ranking should be calculated according to the following rules:

-   The scores should be ranked from the highest to the lowest.
-   If there is a tie between two scores, both should have the same ranking.
-   After a tie, the next ranking number should be the next consecutive integer value. In other words, there should be no holes between ranks.

Return the result table ordered by  `score`  in descending order.

In [0]:
data = [[1, 3.5], [2, 3.65], [3, 4.0], [4, 3.85], [5, 4.0], [6, 3.65]]
columns = ['id', 'score']
schema = {'id':'Int64', 'score':'Float64'}

In [0]:
import pandas

scores_pandas_dataframe = pandas.DataFrame(data=data, columns=columns).astype(schema)

In [0]:
from pyspark.sql import SparkSession

spark_context = SparkSession.builder.appName("LeetCode SQL").getOrCreate()

# Spark DataFrame using Pandas DataFrame
scores_dataframe = spark_context.createDataFrame(scores_pandas_dataframe)
scores_dataframe.printSchema()
scores_dataframe.show(5)

root
 |-- id: long (nullable = true)
 |-- score: double (nullable = true)

+---+-----+
| id|score|
+---+-----+
|  1|  3.5|
|  2| 3.65|
|  3|  4.0|
|  4| 3.85|
|  5|  4.0|
+---+-----+
only showing top 5 rows



In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank

window_spec = Window.orderBy(scores_dataframe.score.desc())
scores_dataframe = scores_dataframe.withColumn("score_rank", dense_rank().over(window_spec))
scores_dataframe.orderBy(scores_dataframe.score_rank, ascending=[False]).show()

+---+-----+----------+
| id|score|score_rank|
+---+-----+----------+
|  1|  3.5|         4|
|  2| 3.65|         3|
|  6| 3.65|         3|
|  4| 3.85|         2|
|  3|  4.0|         1|
|  5|  4.0|         1|
+---+-----+----------+

